## Analisi della distribuzione M/F nei dataset per genere testuale

In questo notebook analizziamo:
1. la distribuzione dei generi (F/M) nei testi contenuti nei file originali, per i generi testuali: **Children**, **Diary**, **Journalism** e **Twitter**.
2. la lunghezza dei testi, misurata in **numero di parole** e **numero di caratteri**, per ciascun genere testuale



### Distribuzione dei generi (F/M)

In [1]:
import re
from collections import Counter
import pandas as pd

In [2]:
# File originali
files = {
    "Children": "../data/dataset_originale/Training/GxG_Children.txt",
    "Diary": "../data/dataset_originale/Training/GxG_Diary.txt",
    "Journalism": "../data/dataset_originale/Training/GxG_Journalism.txt",
    "Twitter": "../data/dataset_originale/Training/GxG_Twitter.txt"
}

In [3]:
# Funzione per contare M/F
def extract_gender_counts(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
        genders = re.findall(r'gender="(F|M)"', content)
        return Counter(genders)

In [4]:
# Raccolta dati
data = []
for genre, path in files.items():
    counts = extract_gender_counts(path)
    data.append({
        "Genere testuale": genre,
        "F": counts.get("F", 0),
        "M": counts.get("M", 0),
        "Totale": counts.get("F", 0) + counts.get("M", 0)
    })

In [5]:
df = pd.DataFrame(data)
df["%F"] = (df["F"] / df["Totale"] * 100).round(2)
df["%M"] = (df["M"] / df["Totale"] * 100).round(2)

In [6]:
df.sort_values("Genere testuale", inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Genere testuale,F,M,Totale,%F,%M
0,Children,100,100,200,50.0,50.0
1,Diary,100,100,200,50.0,50.0
2,Journalism,100,100,200,50.0,50.0
3,Twitter,3000,3000,6000,50.0,50.0


### Lunghezza dei testi

In [9]:
# Estrazione testi <doc> da ciascun file
def extract_text_lengths(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
        matches = re.findall(r'<doc id="\d+" genre="\w+" gender="\w">([\s\S]*?)</doc>', content)
        lengths = []
        for text in matches:
            text = text.strip()
            num_chars = len(text)
            num_words = len(text.split())
            num_sentences = len(re.findall(r"[.!?]+", text))
            lengths.append((num_words, num_chars, num_sentences))
        return lengths

In [11]:
# Calcolo lunghezza media e varianza
results = []

for genre, path in files.items():
    lengths = extract_text_lengths(path)
    if not lengths:
        continue
    word_lengths = [w for w, c, s in lengths]
    char_lengths = [c for w, c, s in lengths]
    sentence_counts = [s for w, c, s in lengths]

    results.append({
        "Genere testuale": genre,
        "Media parole": round(pd.Series(word_lengths).mean(), 2),
        "Varianza parole": round(pd.Series(word_lengths).var(), 2),
        "Media caratteri": round(pd.Series(char_lengths).mean(), 2),
        "Varianza caratteri": round(pd.Series(char_lengths).var(), 2),
        "Media frasi": round(pd.Series(sentence_counts).mean(), 2),
        "Varianza frasi": round(pd.Series(sentence_counts).var(), 2),
        "Numero testi": len(lengths)
    })

df_lunghezze_frasi = pd.DataFrame(results).sort_values("Genere testuale").reset_index(drop=True)
df_lunghezze_frasi

,Genere testuale,Media parole,Varianza parole,Media caratteri,Varianza caratteri,Media frasi,Varianza frasi,Numero testi
0,Children,329.54,31181.90,1903.24,1083110.28,16.86,139.22,200
1,Diary,413.88,102623.12,2469.52,3642663.30,22.10,268.64,200
2,Journalism,565.85,40740.54,3549.89,1642115.61,26.71,141.56,200
3,Twitter,16.90,24.79,117.57,1553.33,1.62,1.65,6000


## Analisi Stanza: distribuzione di aggettivi e verbi con genere morfologico per autore (F/M) e genere testuale

Analisi morfosintattica dei testi GxG (escluso YouTube) per confrontare l'uso di aggettivi e verbi con genere grammaticale (maschile o femminile) tra autori di genere F e M, per ciascun genere testuale.

- Per ciascun documento vengono estratte UPOS + feature morfosintattiche
- Filtrati solo gli aggettivi e i verbi con accordo di genere
- Contate le occorrenze per ciascun genere testuale

In [ ]:
import stanza
from tqdm import tqdm

In [ ]:
stanza.download("it")
nlp = stanza.Pipeline(lang="it", processors="tokenize,mwt,pos,lemma,depparse", use_gpu=False)

In [ ]:
# Analizza tutti i testi e raccoglie conteggi ADJ/VERB con Gender per autori F e M
counts = defaultdict(lambda: {"F": 0, "M": 0})

for genre, path in files.items():
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
        matches = re.findall(r'<doc id="\d+" genre="\w+" gender="(F|M)">([\s\S]*?)</doc>', content)

    for gender, text in tqdm(matches, desc=genre):
        doc = nlp(text.strip())
        for sent in doc.sentences:
            for word in sent.words:
                feats = word.feats if word.feats else ""
                if word.upos in ["ADJ", "VERB"] and ("Gender=Fem" in feats or "Gender=Masc" in feats): #<-- selezione su accordo di genere
                    key = (genre, word.upos)
                    counts[key][gender] += 1

In [ ]:
# Trasforma in DataFrame
rows = []
for (genre, pos), d in counts.items():
    F = d["F"]
    M = d["M"]
    tot = F + M
    rows.append({
        "Genere testuale": genre,
        "Categoria": pos,
        "F": F,
        "M": M,
        "Totale": tot,
        "%F": round(F / tot * 100, 1) if tot > 0 else 0.0,
        "%M": round(M / tot * 100, 1) if tot > 0 else 0.0
    })

In [ ]:
df = pd.DataFrame(rows).sort_values(["Genere testuale", "Categoria"]).reset_index(drop=True)
df

| Genere testuale | Categoria |   F   |   M   | Totale |  %F  |  %M  |
|-----------------|-----------|-------|-------|--------|------|------|
| Children        | ADJ       | 1625  | 1210  |  2835  | 57.3 | 42.7 |
| Children        | VERB      | 1134  | 1022  |  2156  | 52.6 | 47.4 |
| Diary           | ADJ       | 1958  | 2153  |  4111  | 47.6 | 52.4 |
| Diary           | VERB      | 1261  | 1163  |  2424  | 52.0 | 48.0 |
| Journalism      | ADJ       | 2579  | 2911  |  5490  | 47.0 | 53.0 |
| Journalism      | VERB      | 1831  | 1967  |  3798  | 48.2 | 51.8 |
| Twitter         | ADJ       | 1721  | 1803  |  3524  | 48.8 | 51.2 |
| Twitter         | VERB      | 1015  | 1230  |  2245  | 45.2 | 54.8 |

La tabella mostra il numero di aggettivi (ADJ) e verbi (VERB) con marcatura di genere morfologico, prodotti da autori F e M in ciascun genere testuale. Le colonne %F e %M indicano la distribuzione relativa.